# Geodesic Runtime Comparison

In [63]:
import pickle

from tabulate import tabulate

In [64]:
path = '../timing/'
manifold = "Sphere"

## Printing Table Values

In [65]:
save_path = ''.join((path, manifold, '.pkl'))
with open(save_path, 'rb') as f:
    estimates = pickle.load(f)

In [66]:
dims = list(estimates.keys())

mu_time = [estimates[key3][key2]['mu_time'] for key3,v3 in estimates.items() for key2,v2 in v3.items()]
std_time = [estimates[key3][key2]['std_time'] for key3,v3 in estimates.items() for key2,v2 in v3.items()]
length = [estimates[key3][key2]['length'] for key3,v3 in estimates.items()for key2,v2 in v3.items()]
iter_num = [estimates[key3][key2]['iterations'] for key3,v3 in estimates.items() for key2,v2 in v3.items()]
grad_norm = [estimates[key3][key2]['grad_norm'] for key3,v3 in estimates.items() for key2,v2 in v3.items()]
max_iter = [estimates[key3][key2]['max_iter'] for key3,v3 in estimates.items() for key2,v2 in v3.items()]
tol = [estimates[key3][key2]['tol'] for key3,v3 in estimates.items() for key2,v2 in v3.items()]
method = [key2 for key3,v3 in estimates.items() for key2,v2 in v3.items()]

N_methods = len(list(set(method)))
dims = sum([[d]*N_methods for d in dims], [])

In [67]:
names = [r"Dim", r"Method", 
        r"Length", r"Iterations", r"grad_norm", r"mean time",
        r"std time", r"max_iter", r"tol"]

table_dict = {}
table_dict['Dim'] = dims
table_dict['Method'] = method
table_dict['Length'] = length
table_dict['Iterations'] = iter_num
table_dict['grad_norm'] = grad_norm
table_dict['mu_time'] = mu_time
table_dict['std_time'] = std_time
table_dict['max_iter'] = max_iter
table_dict['tol'] = tol

print(tabulate(table_dict, headers=names, floatfmt=".4f"))


  Dim  Method              Length    Iterations    grad_norm    mean time    std time    max_iter     tol
-----  ----------------  --------  ------------  -----------  -----------  ----------  ----------  ------
    2  gradient            2.3827     1000.0000       0.0079       7.5978      0.0976        1000  0.0001
    2  init                2.3971                                                            1000  0.0001
    2  ground_truth        2.3005                                                            1000  0.0001
    2  geodesic_control    2.2971        9.0000       0.0001       0.0895      0.0024        1000  0.0001
    2  adam                2.3002     1000.0000       0.0874       3.1400      0.0203        1000  0.0001
